## Trajectories for m-miba

In [3]:

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go


from scipy.spatial.transform import Rotation as R

from typing import Tuple, List

# After installing ipympl and restarting backend
%matplotlib widget

## Generate Contact Points

https://docs.google.com/presentation/d/1ZvxPcxnbvVUdstRq2_Qzf5mGeaqTVxV470IyfK_p_ng/edit?usp=sharing

In [17]:
#parameters all in mm
OFFSET_X = -3
OFFSET_Y = 0
OFFSET_Z = 2.5


RANGE_X = [-0, 0]   #full range = -33, 33
RANGE_Y = [-0, 0]   #full range = -6, 6
RANGE_Z = [0,1]


NUM_SAMPLES_X = 1   
NUM_SAMPLES_Y = 1
NUM_SAMPLES_Z = 1
X_PLOTTING_RANGE = [-17/4.0,17/4.0]
Y_PLOTTING_RANGE = [-71/4.0,71/4.0]
Z_PLOTTING_RANGE = [-4,1]


dx = 1 # distance in mm between points along each ray of the asterisk
N1 = 1  # number of pts along each ray of the asterisk (not including center point)
N2 = 1  # number of equally spaced rays of the asterisk

In [5]:
# sample_random_points()


In [6]:

def sample_ordered_points(ranges=[[0, 1], [0, 1], [0, 1]], 
                            num_samples=[10, 10, 10], 
                            offsets=[0, 0, 0]):
    """
    Generates 3D points within the specified x, y, z ranges.

    Args:
        ranges (list): List of ranges for x, y, z axes.
                       Format: [[x_start, x_end], [y_start, y_end], [z_start, z_end]]
        num_samples (list): List of the number of samples for x, y, z axes.
                            Format: [num_samples_x, num_samples_y, num_samples_z]
        offsets (list): List of offsets to apply to each axis [x_offset, y_offset, z_offset].

    Returns:
        np.array: Array of shape (num_samples_x * num_samples_y * num_samples_z, 3)
                  containing (x, y, z) points.
    """

    # Unpack range and sample settings for x, y, z
    range_x, range_y, range_z = ranges
    num_x, num_y, num_z = num_samples
    offset_x, offset_y, offset_z = offsets

    # Generate points for each axis, including offsets
    x_points = np.linspace(range_x[0], range_x[1], num_x) + offset_x
    y_points = np.linspace(range_y[0], range_y[1], num_y) + offset_y
    z_points = np.linspace(range_z[0], range_z[1], num_z) + offset_z

    # Create an empty list to store the points with the flipping applied
    flipped_points = []

    # Iterate over each y index to flip x samples
    for i, y in enumerate(y_points):
        # Flip x_points for every other y index
        if i % 2 == 0:
            current_x_points = x_points
        else:
            current_x_points = x_points[::-1]  # Reverse the order

        # Create the grid for the current y index
        x_grid, z_grid = np.meshgrid(current_x_points, z_points, indexing='ij')
        
        # Flatten the grid and combine with the current y value
        current_points = np.column_stack((x_grid.ravel(), np.full(x_grid.size, y), z_grid.ravel()))
        
        # Append to the list of flipped points
        flipped_points.append(current_points)

    # Concatenate all flipped points into a single array
    p_sensor_contactpt = np.vstack(flipped_points)

    ## Map for m-miba mount (z pointing down) - corrects the orientation 
    # z -> -z
    T_zMount_sensor = np.array([[1, 0, 0], [0, 1, 0], [0, 0, -1]])
    p_zMount_contactpt = (T_zMount_sensor @ p_sensor_contactpt.T).T
    
    return p_zMount_contactpt



In [7]:
def plot_points(points, plotting_ranges):
    """
    Plots the contact points in 2D (x, y) and 3D (x, y, z) with specified axis ranges.
    
    Args:
        points (np.array): Array of shape (N, 3) with (x, y, z) points.
        plotting_ranges (list): [x_range, y_range, z_range]
    """

    x_plotting_range, y_plotting_range, z_plotting_range = plotting_ranges
    
    # 2D plot (x, y)
    # x -> -x for viewing purposes
    fig_2d = px.scatter(x=-points[:, 0], y=points[:, 1], title='2D Plot of (x, y) Points')
    fig_2d.update_xaxes(title='x', range=x_plotting_range, scaleanchor="y", scaleratio=1)
    fig_2d.update_yaxes(title='y', range=y_plotting_range, scaleanchor="x", scaleratio=1)
    fig_2d.show()


    
    # 3D plot (x, y, z)
    aspect_ratio = dict(x=1, y=1, z=1)

    fig_3d = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(size=3, color=points[:, 2], colorscale='Viridis', opacity=0.8)
    )])
    fig_3d.update_layout(scene=dict(
        xaxis=dict(title='x', range=x_plotting_range),
        yaxis=dict(title='y', range=y_plotting_range),
        zaxis=dict(title='z', range=z_plotting_range),
        aspectmode="manual", aspectratio=aspect_ratio),
        title='3D Plot of (x, y, z) Points')
    fig_3d.show()


In [18]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])

plot_points(points, plotting_ranges = [X_PLOTTING_RANGE, Y_PLOTTING_RANGE, Z_PLOTTING_RANGE])



## Create A Full Trajectory
We take the points generated and convert them to a continuous trajectory of points comprising of:

1. the point to be sampled 
2. the location to retract to after sampling a point
3. the location to start the the approach to for the the next sample point (this is the same as the point to retract to)

The I_tare flag is added to the approach points to indicate when to tare the collected data. 

The t_dwell time indicates how long to stay at the given point.

The final trajectory is a an array of N points to traverse where each row is a list of 7 values:

$$(x, y, z, \theta, \phi, T_{dwell}, \mathbb{I}_{contact})



In [418]:

def convert_mountPoints_to_robot_coordinates(points):
    # Number of rows (points) in p_gantry_contactPoint
    N = points.shape[0]

    # Create theta and phi columns filled with zeros
    theta = np.zeros((N, 1))  # Column vector of zeros with shape (N, 1)
    phi = np.zeros((N, 1))    # Column vector of zeros with shape (N, 1)
    raw_commands = np.hstack((points, theta, phi))
    return raw_commands


def make_approach_points(points, approach_offset_from_center):
    N = points.shape[0]
    z_approach_points = approach_offset_from_center*np.ones((N,1))
    approach_points = np.hstack((points[:,0:1],points[:,1:2], z_approach_points))
    return approach_points

def broadcast_t_dwell(coordinate, t_dwell):
    return np.hstack((coordinate, np.ones((coordinate.shape[0], 1)) * t_dwell))

def broadcast_I_tare(coordinate, I_tare):
    return np.hstack((coordinate, np.ones((coordinate.shape[0], 1)) * I_tare))

def broadcast_contact(coordinate, contact):
    return np.hstack((coordinate, np.expand_dims(np.arange(1, coordinate.shape[0]+1, dtype = "int").T * contact, 1)))

def convert_points_to_trajectory(points):

    approach_offset_from_center = 6  # Must be greater than traversal height
    
    mmiba_coordinates = []
    coordinate_length = 8 #[x, y, z, theta, phi, t_dwell, I_tare_flag, contact point]
    
    p_mount_approach = make_approach_points(points, approach_offset_from_center)

    contact_coordinates = convert_mountPoints_to_robot_coordinates(points)
    approach_coordinates = convert_mountPoints_to_robot_coordinates(p_mount_approach)
    retract_coordinates = convert_mountPoints_to_robot_coordinates(p_mount_approach)

    # Time to sample = 1s
    t_dwell_contact = 60
    t_dwell_approach_retract = 0.7
    t_dwell_home = 60

    #wait time for each point
    contact_coordinates = broadcast_t_dwell(contact_coordinates, t_dwell=t_dwell_contact)
    approach_coordinates = broadcast_t_dwell(approach_coordinates, t_dwell_approach_retract)
    retract_coordinates = broadcast_t_dwell(retract_coordinates, t_dwell_approach_retract) 

    #flag to indicate when to tare the ATI sensor values    
    contact_coordinates = broadcast_I_tare(contact_coordinates, 0)
    approach_coordinates = broadcast_I_tare(approach_coordinates, 1)
    retract_coordinates = broadcast_I_tare(retract_coordinates, 0)

    #add contact point index to each contact coordinate
    contact_coordinates = broadcast_contact(contact_coordinates,1)
    approach_coordinates = broadcast_contact(approach_coordinates,0)
    retract_coordinates = broadcast_contact(retract_coordinates,0)

    #merge approach, contact, and retract coordinates
    merged_coordinates = np.zeros((contact_coordinates.shape[0] * 3, coordinate_length))
    merged_coordinates[::3] = approach_coordinates
    merged_coordinates[1::3] = contact_coordinates
    merged_coordinates[2::3] = retract_coordinates
    

    # Go to home at end
    home_coordinate = [0,0,approach_offset_from_center, 0, 0, t_dwell_home, 0, 0]
    merged_coordinates = np.vstack((merged_coordinates, home_coordinate))
    print("trajectory size: ",merged_coordinates.shape)

    mmiba_coordinates.append(merged_coordinates)
    return mmiba_coordinates




In [419]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])

mmibaTrajectory = convert_points_to_trajectory(points) 


mmibaTrajectory = mmibaTrajectory[0]

idx = 2
description = "single_press"
save_traj_path = "trajectories/mmibaTrajectory" + description + f"_{idx}.npy"
np.save(save_traj_path, mmibaTrajectory)

trajectory size:  (4, 8)


## Create a Trajectory for Shear
1. probe target point
2. probe shear point 1
3. probe shear point 2 
.
.
.




In [ ]:
def generate_shear_points(dx, N1, N2):
    ray_length = 2 * N1
    rays = np.zeros((3, ray_length))

    # Generate points along a single ray in both directions from the center
    for i in range(N1):
        offset = dx * (i + 1)
        rays[:, i] = [offset, 0, 0]
        rays[:, N1 + i] = [-offset, 0, 0]

    # Initialize the pattern array with a central point and rotated rays
    pattern_length = 1 + ray_length * N2
    pattern = np.zeros((3, pattern_length))
    pattern[:, 0] = [0, 0, 0]  # Center point

    # Rotate rays around the z-axis to create the shear pattern
    for j in range(N2):
        angle = j * (360 / N2)
        rot_mat = np.array([[np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0],
                            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0],
                            [0, 0, 1]])
        rotated_rays = rot_mat @ rays
        pattern[:, 1 + j * ray_length : 1 + (j + 1) * ray_length] = rotated_rays
    
    pattern = pattern.T
    return pattern

def generate_shear_trajectory(shear_coordinates,contact_coordinates,approach_offset_from_center,t_dwell_contact,offsets):
    ## Map for m-miba mount (z pointing down) - corrects the orientation 
    # z -> -z
    T_zMount_sensor = np.array([[1, 0, 0], [0, 1, 0], [0, 0, -1]])
    offsets = np.array(offsets)
    offsets = (T_zMount_sensor @ offsets.T).T
    
    
    traj_pattern = []
    center = np.array([shear_coordinates[0,0], shear_coordinates[0,1], shear_coordinates[0,2],0, 0, 0.5,0])
    approach_point = [0 - offsets[0], 0 - offsets[1], approach_offset_from_center - offsets[2],0,0,0.5,1]
    retract_point =  [0 - offsets[0], 0 - offsets[1], approach_offset_from_center - offsets[2],0,0,0.5,0]
    for point in shear_coordinates[1:,:]:
        shear_point = [point[0],point[1],point[2],0,0,t_dwell_contact,0]
        shear_point_traj = np.vstack((approach_point,center, shear_point, center))
        traj_pattern.append(shear_point_traj)
    traj_pattern.append(retract_point)
    traj_pattern = np.vstack(traj_pattern)
    traj_pattern = traj_pattern[1:,:]

    print("traj_pattern: ", traj_pattern)

    #trajectory of points including approach
    traj_patterns = []

    #zero out last two columns (the I tare and the tdwell) because we don't want to add those
    contact_coordinates[:,-2:] = 0

    print("contact_coordinates: ",contact_coordinates)
    for center in contact_coordinates:
        translated_traj_pattern = traj_pattern + center 
        traj_patterns.append(translated_traj_pattern) 
    # Stack all trajectory patterns vertically
    traj_patterns = np.vstack(traj_patterns)
    
    return traj_patterns, traj_pattern.shape[0]

    
def convert_points_to_trajectory_shear(points,dx,N1,N2,offsets):

    approach_offset_from_center = 6 # Must be greater than traversal height
    
    mmiba_coordinates = []
    coordinate_length = 8 #[x, y, z, theta, phi, t_dwell, I_tare_flag, contact point]
    
    p_mount_approach = make_approach_points(points, approach_offset_from_center)

    shear_coordinates = generate_shear_points(dx, N1, N2) #generate just the simple pattern
    contact_coordinates = convert_mountPoints_to_robot_coordinates(points) #add the raw, pitch values
    approach_coordinates = convert_mountPoints_to_robot_coordinates(p_mount_approach)
    retract_coordinates = convert_mountPoints_to_robot_coordinates(p_mount_approach)

    # Time to sample = 1s
    t_dwell_contact = 1

    contact_coordinates = broadcast_t_dwell(contact_coordinates, t_dwell=t_dwell_contact)
    approach_coordinates = broadcast_t_dwell(approach_coordinates, 0.5)
    retract_coordinates = broadcast_t_dwell(retract_coordinates, 0.5) 
    
    contact_coordinates = broadcast_I_tare(contact_coordinates, 0)
    approach_coordinates = broadcast_I_tare(approach_coordinates, 1)
    retract_coordinates = broadcast_I_tare(retract_coordinates, 0)

    #add contact point index to each contact coordinate
    contact_coordinates = broadcast_contact(contact_coordinates,1)  #i * length of pattern + 1
    approach_coordinates = broadcast_contact(approach_coordinates,0)
    retract_coordinates = broadcast_contact(retract_coordinates,0)

    shear_coordinates, shear_group_length = generate_shear_trajectory(shear_coordinates, contact_coordinates, approach_offset_from_center, t_dwell_contact,offsets)

    print("shear_coordinates shape: ",shear_coordinates.shape)
    
    merged_coordinates = np.zeros((contact_coordinates.shape[0] * (3 + shear_group_length), coordinate_length))
    merged_coordinates[::3 + shear_group_length] = approach_coordinates
    merged_coordinates[1::3 + shear_group_length] = contact_coordinates
    merged_coordinates[2::3 + shear_group_length] = retract_coordinates
    
    
    for i in range(len(approach_coordinates)):
        start_idx = 3 + i * (3 + shear_group_length)
        shear_start_idx = i * shear_group_length
        merged_coordinates[start_idx:start_idx + shear_group_length] = shear_coordinates[shear_start_idx: shear_start_idx + shear_group_length]

    # Go to home at end
    home_coordinate = [0,0,approach_offset_from_center, 0, 0, 0, 0, 0]
    merged_coordinates = np.vstack((merged_coordinates, home_coordinate))
    print(merged_coordinates.shape)



    mmiba_coordinates.append(merged_coordinates)
    
    return mmiba_coordinates

In [380]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])

mmibaTrajectory_shear = convert_points_to_trajectory_shear(points,dx,N1,N2,offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z]
)


mmibaTrajectory_shear = mmibaTrajectory_shear[0]

idx = 2
description = "shear"
save_traj_path_shear = "trajectories/mmibaTrajectory" + description + f"_{idx}.npy"
np.save(save_traj_path_shear, mmibaTrajectory_shear)



traj_pattern:  [[ 0.   0.   0.   0.   0.   0.5  0. ]
 [ 1.   0.   0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   0.   0.5  0. ]
 [ 0.  27.   8.5  0.   0.   0.5  1. ]
 [ 0.   0.   0.   0.   0.   0.5  0. ]
 [-1.   0.   0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   0.   0.5  0. ]
 [ 0.  27.   8.5  0.   0.   0.5  0. ]]
contact_coordinates:  [[-28.   -1.   -2.5   0.    0.    1.    0.    0. ]
 [-27.5  -1.   -2.5   0.    0.    1.    0.    0. ]
 [-27.   -1.   -2.5   0.    0.    1.    0.    0. ]
 [-26.5  -1.   -2.5   0.    0.    1.    0.    0. ]
 [-26.   -1.   -2.5   0.    0.    1.    0.    0. ]
 [-26.   -0.5  -2.5   0.    0.    1.    0.    0. ]
 [-26.5  -0.5  -2.5   0.    0.    1.    0.    0. ]
 [-27.   -0.5  -2.5   0.    0.    1.    0.    0. ]
 [-27.5  -0.5  -2.5   0.    0.    1.    0.    0. ]
 [-28.   -0.5  -2.5   0.    0.    1.    0.    0. ]
 [-28.    0.   -2.5   0.    0.    1.    0.    0. ]
 [-27.5   0.   -2.5   0.    0.    1.    0.    0. ]
 [-27.    0.   -2.5   0.    0.    1.    0.    

ValueError: operands could not be broadcast together with shapes (8,7) (8,) 

In [183]:
def generate_shear_points_plotting(points, dx, N1, N2):
    """
    Generate shear points around a given center.

    Parameters:
    points (array): Initial points (unused in current function logic).
    dx (float): Step size along each ray.
    N1 (int): Number of points per ray (in each direction from the center).
    N2 (int): Number of rays around the center.

    Returns:
    all_pattern (array): 
    """
    ray_length = 2 * N1
    rays = np.zeros((3, ray_length))

    # Generate points along a single ray in both directions from the center
    for i in range(N1):
        offset = dx * (i + 1)
        rays[:, i] = [offset, 0, 0]
        rays[:, N1 + i] = [-offset, 0, 0]

    # Initialize the pattern array with a central point and rotated rays
    pattern_length = 1 + ray_length * N2
    pattern = np.zeros((3, pattern_length))
    pattern[:, 0] = [0, 0, 0]  # Center point

    # Rotate rays around the z-axis to create the shear pattern
    for j in range(N2):
        angle = j * (360 / N2)
        rot_mat = np.array([[np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0],
                            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0],
                            [0, 0, 1]])
        rotated_rays = rot_mat @ rays
        pattern[:, 1 + j * ray_length : 1 + (j + 1) * ray_length] = rotated_rays
    
    pattern = pattern.T
    

    #there is probably a better place to put this
    #create shear portion of trajectory
    approach_offset_from_center = 10
    traj_pattern = []
    center = pattern[0,:]
    for point in pattern[1:,:]:
        approach_point = [0,0,-approach_offset_from_center,]
        retract_point = [0,0,-approach_offset_from_center]
        shear_point_traj = np.vstack((approach_point,center, point, center, retract_point))
        traj_pattern.append(shear_point_traj)
    traj_pattern = np.vstack(traj_pattern)

    #which points we are sampling just for plotting purposes
    all_patterns = []
    for center in points:
        translated_pattern = pattern + center
        all_patterns.append(translated_pattern) 
    # Stack all patterns vertically
    all_patterns = np.vstack(all_patterns)

    #trajectory of points including approach
    traj_patterns = []
    for center in points:
        translated_traj_pattern = traj_pattern + center
        traj_patterns.append(translated_traj_pattern) 
    # Stack all trajectory patterns vertically
    traj_patterns = np.vstack(traj_patterns)


    fig_3d = go.Figure(data=[go.Scatter3d(
    x=pattern[:, 0],
    y=pattern[:, 1],
    z=pattern[:, 2],
    mode='markers',
    marker=dict(size=3, color=points[:, 2], colorscale='Viridis', opacity=0.8)
    )])
    fig_3d.show()

    fig_3d_all = go.Figure(data=[go.Scatter3d(
    x=all_patterns[:, 0],
    y=all_patterns[:, 1],
    z=all_patterns[:, 2],
    mode='markers',
    marker=dict(size=3, opacity=0.8)
    )])
    fig_3d_all.show() 



In [ ]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])

generate_shear_points_plotting(points,dx,N1,N2)

## OLD CODE

In [ ]:
#old sample ordered points code

def sample_ordered_points(ranges=[[0, 1], [0, 1], [0, 1]], 
                            num_samples=[10, 10, 10], 
                            offsets=[0, 0, 0]):
    """
    Generates 3D points within the specified x, y, z ranges.

    Args:
        ranges (list): List of ranges for x, y, z axes.
                       Format: [[x_start, x_end], [y_start, y_end], [z_start, z_end]]
        num_samples (list): List of the number of samples for x, y, z axes.
                            Format: [num_samples_x, num_samples_y, num_samples_z]
        offsets (list): List of offsets to apply to each axis [x_offset, y_offset, z_offset].

    Returns:
        np.array: Array of shape (num_samples_x * num_samples_y * num_samples_z, 3)
                  containing (x, y, z) points.
    """
    # # Unpack range and sample settings for x, y, z
    # range_x, range_y, range_z = ranges
    # num_x, num_y, num_z = num_samples
    # offset_x, offset_y, offset_z = offsets

    # # Generate points for each axis, including offsets
    # x_points = np.linspace(range_x[0], range_x[1], num_x) + offset_x
    # y_points = np.linspace(range_y[0], range_y[1], num_y) + offset_y
    # z_points = np.linspace(range_z[0], range_z[1], num_z) + offset_z

    # # Create meshgrid for x, y, and z and reshape to a list of (x, y, z) points
    # x_grid, y_grid, z_grid = np.meshgrid(x_points, y_points, z_points, indexing='ij')
    # p_sensor_contactpt = np.column_stack((x_grid.ravel(), y_grid.ravel(), z_grid.ravel()))

    # # Unpack range and sample settings for x, y, z
    range_x, range_y, range_z = ranges
    num_x, num_y, num_z = num_samples
    offset_x, offset_y, offset_z = offsets

    # Generate points for each axis, including offsets
    x_points = np.linspace(range_x[0], range_x[1], num_x) + offset_x
    y_points = np.linspace(range_y[0], range_y[1], num_y) + offset_y
    z_points = np.linspace(range_z[0], range_z[1], num_z) + offset_z

    # Create an empty list to store the points with the flipping applied
    flipped_points = []

    # Iterate over each y index to flip x samples
    for i, y in enumerate(y_points):
        # Flip x_points for every other y index
        if i % 2 == 0:
            current_x_points = x_points
        else:
            current_x_points = x_points[::-1]  # Reverse the order

        # Create the grid for the current y index
        x_grid, z_grid = np.meshgrid(current_x_points, z_points, indexing='ij')
        
        # Flatten the grid and combine with the current y value
        current_points = np.column_stack((x_grid.ravel(), np.full(x_grid.size, y), z_grid.ravel()))
        
        # Append to the list of flipped points
        flipped_points.append(current_points)

    # Concatenate all flipped points into a single array
    p_sensor_contactpt = np.vstack(flipped_points)

    #*********************
    # range_x, range_y, range_z = ranges
    # num_x, num_y, num_z = num_samples
    # offset_x, offset_y, offset_z = offsets

    # # Generate points for each axis, including offsets
    # x_points = np.linspace(range_x[0], range_x[1], num_x) + offset_x
    # y_points = np.linspace(range_y[0], range_y[1], num_y) + offset_y
    # z_points = np.linspace(range_z[0], range_z[1], num_z) + offset_z

    # # Create an empty list to store the points with the zigzag pattern applied
    # zigzag_points = []

    # # Iterate over each x index to flip y samples
    # for i, x in enumerate(x_points):
    #     # Flip y_points for every other x index
    #     if i % 2 == 0:
    #         current_y_points = y_points
    #     else:
    #         current_y_points = y_points[::-1]  # Reverse the order

    #     # Create the grid for the current x index
    #     y_grid, z_grid = np.meshgrid(current_y_points, z_points, indexing='ij')
        
    #     # Flatten the grid and combine with the current x value
    #     current_points = np.column_stack((np.full(y_grid.size, x), y_grid.ravel(), z_grid.ravel()))
        
    #     # Append to the list of zigzag points
    #     zigzag_points.append(current_points)

    # # Concatenate all zigzag points into a single array
    # p_sensor_contactpt = np.vstack(zigzag_points)
    #******************************

    ## Map for m-miba mount (z pointing down) - corrects the orientation 
 
    # z -> -z
    T_zMount_sensor = np.array([[1, 0, 0], [0, 1, 0], [0, 0, -1]])
    p_zMount_contactpt = (T_zMount_sensor @ p_sensor_contactpt.T).T
    
    return p_zMount_contactpt

